In [26]:
import os
from json import load
import numpy as np
from PIL import Image

# 各種関数の定義
- load_image: 画像を読み込んで中心をクロップし、224,224にリサイズしたPILデータを返す
- get_data: テストデータと学習データの一覧をJSONファイルから読み込む 猫：１、犬：０
- save_crop_image: 中心をクロップしたイメージを./simple_crop/(test_data or train_data)/imagesに保存し、  
それらのファイル名とラベルを./simple_crop/(test_data or train_data)/(test or train) \_labeled.csvとして保存する  
(ChainerのLabeledImageDataset用)
    

In [30]:
def load_image(file, size=(224,224)):
    path='./images/'+file
    img=Image.open(path)
    #３次元配列じゃないのが混ざっているのでとりあえず全部RGBにする
    img=img.convert('RGB')
    w,h=img.size
    w_c, h_c=w//2, h//2
    half_short_side=min(w_c, h_c)
    x0=w_c-half_short_side
    y0=h_c-half_short_side
    x1=w_c+half_short_side
    y1=h_c+half_short_side
    img=img.crop((x0,y0,x1,y1))
    img=img.resize(size)
    return img

def get_data(kind):
    path='./'+kind+'.json'
    with open (path) as f:
        df=load(f)
    
    data=[]
    for k0, v0 in df.items():
        for k1, v1 in df[k0].items():
            for file in df[k0][k1]['images']:
                file=file+'.jpg'
                if k0=='cat':
                    data.append([file, 1])
                else:
                    data.append([file, 0])
    return np.array(data)

def save_crop_image(data, kind):
    path='./simple_crop/'+kind+'/'
    dir_path=path+'images'
    os.makedirs(dir_path, exist_ok=True)
    for file, label in data:
        img=load_image(file)
        file_path=path+'images'+file
        label_path=path+kind+'_labeled.csv'
        img.save(file_path)
        with open (label_path, 'a') as f:
            added_sentence=file+' '+str(label)+'\n'
            f.write(added_sentence)

In [19]:
# データの読み込み
train_data=get_data('train')
test_data=get_data('test')

複数回使うとデータが重複するのでコメントアウト

In [31]:
#save_crop_image(train_data, 'train_data')
#save_crop_image(test_data, 'test_data')